In [3]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

transactions_train.csv

In [ ]:
transactions = pd.read_csv('./dataset/h&m/transactions_train.csv')
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31788324 entries, 0 to 31788323
Data columns (total 5 columns):
 #   Column            Dtype  
---  ------            -----  
 0   t_dat             object 
 1   customer_id       object 
 2   article_id        int64  
 3   price             float64
 4   sales_channel_id  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 1.2+ GB


In [4]:
transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2


In [5]:
transactions['t_dat'] = pd.to_datetime(transactions['t_dat'])
print("Data Type After Conversion:")
transactions.dtypes

Data Type After Conversion:


t_dat               datetime64[ns]
customer_id                 object
article_id                   int64
price                      float64
sales_channel_id             int64
dtype: object

In [6]:
transactions['year'] = transactions['t_dat'].dt.year
transactions['month'] = transactions['t_dat'].dt.month
transactions['day_of_week'] = transactions['t_dat'].dt.dayofweek
transactions['week_of_year'] = transactions['t_dat'].dt.isocalendar().week

In [7]:
transactions['quarter'] = transactions['t_dat'].dt.quarter
transactions['day'] = transactions['t_dat'].dt.day
transactions['is_weekend'] = transactions['t_dat'].dt.dayofweek >= 5

In [10]:
transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_id,year,month,day_of_week,week_of_year,quarter,day,is_weekend
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2,2018,9,3,38,3,20,False
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2,2018,9,3,38,3,20,False
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2,2018,9,3,38,3,20,False
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2,2018,9,3,38,3,20,False
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2,2018,9,3,38,3,20,False


In [11]:
print("Missing Values per Column:")
transactions.isnull().sum()

Missing Values per Column:


t_dat               0
customer_id         0
article_id          0
price               0
sales_channel_id    0
year                0
month               0
day_of_week         0
week_of_year        0
quarter             0
day                 0
is_weekend          0
dtype: int64

In [12]:
duplicate_transactions = transactions.duplicated().sum()
print(f"Number of duplicate transactions: {duplicate_transactions}")

Number of duplicate transactions: 2974905


In [13]:
transactions = transactions.drop_duplicates()

In [14]:
duplicate_transactions = transactions.duplicated().sum()
print(f"Number of duplicate transactions: {duplicate_transactions}")

Number of duplicate transactions: 0


In [15]:
q1 = transactions['price'].quantile(0.25)
q3 = transactions['price'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr
outliers = transactions[(transactions['price'] < lower_bound) | (transactions['price'] > upper_bound)]
print(f"Number of outliers in price: {len(outliers)}")

Number of outliers in price: 1325119


In [16]:
transactions['log_price'] = np.log1p(transactions['price'])

In [17]:
monthly_sales = transactions.groupby(['year', 'month']).size().reset_index(name='num_transactions')

In [18]:
transactions['repeat_purchase'] = transactions.duplicated(subset=['customer_id', 'article_id'], keep=False)

articles.csv

In [20]:
articles = pd.read_csv('./dataset/h&m/articles.csv')
print(articles.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105542 entries, 0 to 105541
Data columns (total 25 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   article_id                    105542 non-null  int64 
 1   product_code                  105542 non-null  int64 
 2   prod_name                     105542 non-null  object
 3   product_type_no               105542 non-null  int64 
 4   product_type_name             105542 non-null  object
 5   product_group_name            105542 non-null  object
 6   graphical_appearance_no       105542 non-null  int64 
 7   graphical_appearance_name     105542 non-null  object
 8   colour_group_code             105542 non-null  int64 
 9   colour_group_name             105542 non-null  object
 10  perceived_colour_value_id     105542 non-null  int64 
 11  perceived_colour_value_name   105542 non-null  object
 12  perceived_colour_master_id    105542 non-null  int64 
 13 

In [122]:
articles.head()

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."


In [123]:
missing_values = articles.isnull().sum()
print("Missing Values:")
print(missing_values[missing_values > 0])

Missing Values:
detail_desc    416
dtype: int64


In [124]:
articles['detail_desc'].fillna("No description available", inplace=True)

In [125]:
print("Missing Values After Imputation:")
articles.isnull().sum()

Missing Values After Imputation:


article_id                      0
product_code                    0
prod_name                       0
product_type_no                 0
product_type_name               0
product_group_name              0
graphical_appearance_no         0
graphical_appearance_name       0
colour_group_code               0
colour_group_name               0
perceived_colour_value_id       0
perceived_colour_value_name     0
perceived_colour_master_id      0
perceived_colour_master_name    0
department_no                   0
department_name                 0
index_code                      0
index_name                      0
index_group_no                  0
index_group_name                0
section_no                      0
section_name                    0
garment_group_no                0
garment_group_name              0
detail_desc                     0
dtype: int64

customers.csv

In [ ]:
customers = pd.read_csv('./dataset/h&m/customers.csv')
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1371980 entries, 0 to 1371979
Data columns (total 7 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   customer_id             1371980 non-null  object 
 1   FN                      476930 non-null   float64
 2   Active                  464404 non-null   float64
 3   club_member_status      1365918 non-null  object 
 4   fashion_news_frequency  1355969 non-null  object 
 5   age                     1356119 non-null  float64
 6   postal_code             1371980 non-null  object 
dtypes: float64(3), object(4)
memory usage: 73.3+ MB


In [137]:
customers.head()

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,NaN,NaN,ACTIVE,NONE,49.0,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,NaN,NaN,ACTIVE,NONE,25.0,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,NaN,NaN,ACTIVE,NONE,24.0,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,NaN,NaN,ACTIVE,NONE,54.0,5d36574f52495e81f019b680c843c443bd343d5ca5b1c2...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,1.0,1.0,ACTIVE,Regularly,52.0,25fa5ddee9aac01b35208d01736e57942317d756b32ddd...


In [138]:
missing_values = customers.isnull().sum()
print("Missing Values:\n", missing_values[missing_values > 0])

Missing Values:
 FN                        895050
Active                    907576
club_member_status          6062
fashion_news_frequency     16011
age                        15861
dtype: int64


In [139]:
median_age = customers['age'].median()
customers['age'].fillna(median_age, inplace=True)

customers['age_group'] = pd.cut(customers['age'], bins=[0, 18, 30, 50, 100],
                                labels=['Teen', 'Young Adult', 'Middle Aged', 'Senior'])

In [140]:
customers['club_member_status'].fillna('UNKNOWN', inplace=True)
customers['FN'] = customers['FN'].fillna(0).astype(bool)
customers['Active'] = customers['Active'].fillna(0).astype(bool)

In [141]:
customers['fashion_news_frequency'].replace({'NONE': 'None', 'nan': 'None'}, inplace=True)
customers['fashion_news_frequency'].fillna('None', inplace=True)

In [ ]:
transactions.to_csv("./dataset/cleaned_dataset/cleaned_transactions.csv", index=False)
articles.to_csv("./dataset/cleaned_dataset/cleaned_articles.csv", index=False)
customers.to_csv("./dataset/cleaned_dataset/cleaned_customers.csv", index=False)

In [15]:
transactions = pd.read_csv("./dataset/cleaned_dataset/cleaned_transactions.csv")
customers = pd.read_csv("./dataset/cleaned_dataset/cleaned_customers.csv")

In [5]:
transactions["t_dat"] = pd.to_datetime(transactions["t_dat"])
max_date = transactions["t_dat"].max()

In [6]:
rfm_df = transactions.groupby("customer_id").agg(
    Recency=("t_dat", lambda x: (max_date - x.max()).days),  # Days since last purchase
    Frequency=("customer_id", "count"),  # Number of purchases
    Monetary=("price", "sum"),  # Total spending
)

In [7]:
purchase_intervals = transactions.groupby("customer_id")["t_dat"].apply(lambda x: x.diff().mean().days)
rfm_df["Avg_Purchase_Interval"] = purchase_intervals

In [8]:
rfm_df["Avg_Purchase_Interval"].fillna(rfm_df["Recency"], inplace=True)
rfm_df.reset_index(inplace=True)

In [9]:
rfm_df.head()

,customer_id,Recency,Frequency,Monetary,Avg_Purchase_Interval
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,17,19,0.543932,34.0
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,76,78,2.412237,8.0
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,7,15,0.606525,51.0
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,471,2,0.060983,0.0
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,41,13,0.469695,55.0


In [ ]:
rfm_df.to_csv("./dataset/feature_engineering/customers_rfm.csv", index=False)

In [11]:
price_sensitivity = transactions.groupby("customer_id")["price"].agg(["mean", "std"]).rename(
    columns={"mean": "Avg_Price", "std": "Price_StdDev"}
)

price_sensitivity["Price_Sensitivity"] = price_sensitivity["Price_StdDev"] / price_sensitivity["Avg_Price"]
price_sensitivity["Price_Sensitivity"].fillna(0, inplace=True)
price_sensitivity.reset_index(inplace=True)
price_sensitivity.head()

,customer_id,Avg_Price,Price_StdDev,Price_Sensitivity
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0.028628,0.014723,0.514299
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0.030926,0.017524,0.566631
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0.040435,0.017878,0.442153
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0.030492,0.000000,0.000000
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0.036130,0.012638,0.349787


In [ ]:
price_sensitivity.to_csv("./dataset/feature_engineering/customers_price_sensitivity.csv", index=False)

In [ ]:
customers = customers.merge(price_sensitivity[["customer_id", "Price_Sensitivity"]], on="customer_id", how="left")
customers["Price_Sensitivity"].fillna(0, inplace=True)
customers.to_csv("./dataset/feature_engineering/customers_updated.csv", index=False)
customers.head()

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code,age_group,Price_Sensitivity
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,False,False,ACTIVE,NaN,49.0,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...,Middle Aged,0.514299
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,False,False,ACTIVE,NaN,25.0,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...,Young Adult,0.566631
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,False,False,ACTIVE,NaN,24.0,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...,Young Adult,0.442153
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,False,False,ACTIVE,NaN,54.0,5d36574f52495e81f019b680c843c443bd343d5ca5b1c2...,Senior,0.000000
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,True,True,ACTIVE,Regularly,52.0,25fa5ddee9aac01b35208d01736e57942317d756b32ddd...,Senior,0.349787


In [17]:
purchase_frequency = transactions.groupby("customer_id")["article_id"].count().reset_index()
purchase_frequency.columns = ["customer_id", "purchase_frequency"]

In [18]:
avg_purchase_value = transactions.groupby("customer_id")["price"].mean().reset_index()
avg_purchase_value.columns = ["customer_id", "avg_purchase_value"]

In [19]:
customers = customers.merge(purchase_frequency, on="customer_id", how="left")
customers = customers.merge(avg_purchase_value, on="customer_id", how="left")

In [ ]:
customers.fillna(0, inplace=True)
customers.to_csv("./dataset/feature_engineering/customers_with_clustering_features.csv", index=False)

In [21]:
customers.head()

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code,age_group,Price_Sensitivity,purchase_frequency,avg_purchase_value
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,False,False,ACTIVE,0,49.0,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...,Middle Aged,0.514299,19.0,0.028628
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,False,False,ACTIVE,0,25.0,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...,Young Adult,0.566631,78.0,0.030926
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,False,False,ACTIVE,0,24.0,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...,Young Adult,0.442153,15.0,0.040435
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,False,False,ACTIVE,0,54.0,5d36574f52495e81f019b680c843c443bd343d5ca5b1c2...,Senior,0.000000,2.0,0.030492
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,True,True,ACTIVE,Regularly,52.0,25fa5ddee9aac01b35208d01736e57942317d756b32ddd...,Senior,0.349787,13.0,0.036130
